In [ ]:
# todo:
# transmission terms as second order

## Import modules

In [ ]:
import pandas as pd
from reactionmodel.model import Species, Reaction, Model
from reactionmodel.hook import HookAwareModel
import reactionmodel.parser as parser

## Load arrows from the excel document

In [3]:
def concatenate_columns(df, columns, parentheses=True, sep='*', na_rep=None):
    df = df[columns].copy()
    if parentheses:
        df['left'] = '('
        df['right'] = ')'
    else:
        df['left'] = ''
        df['right'] = ''
    for c in columns:
        df[c] = df['left'].str.cat(df[c]).str.cat(df['right'])
    
    first_col = df[columns[0]]
    later_cols = df[columns[1:]]
    return first_col.str.cat(later_cols, sep=sep, na_rep=na_rep)

In [ ]:
arrows_file = pd.ExcelFile('./ModelArrows.xlsx')
arrows = arrows_file.parse('Reactions').drop(0)
arrows = arrows[arrows['Include'].astype(bool)]
arrows['Initial'] = arrows['Initial (A)'].str.cat(arrows['Initial (N)'], sep='_')
arrows['Final'] = arrows['Final (A)'].str.cat(arrows['Final (N)'], sep='_')

arrows['Initial'] = arrows['Initial'].str.replace('-_-', '')
arrows['Final'] = arrows['Final'].str.replace('-_-', '')
# It's dumb, but k stem is guaranteed non-blank where Description (ie not Description stem) is guaranteed non-blank
arrows['k'] = concatenate_columns(arrows, ['k stem', 'k']).fillna(arrows['k stem'])
arrows['Description'] = concatenate_columns(arrows, ['Description stem', 'Description'], sep=' ', parentheses=False).fillna(arrows['Description'])

arrows = arrows[['Group', 'Description', 'Initial', 'Final', 'k', 'trigger']]

arrows

,Group,Description,Initial,Final,k,trigger
1,Colonization,"Colonization of ""healthy"" host by DS without i...",M_M,M_DS,(beta * (1 - CR_N) * (1 - p_beta_infection_M) ...,NaN
2,Colonization,"Colonization of ""healthy"" host by DS with inst...",M_M,DS_DS,(beta * (1 - CR_N) * (p_beta_infection_M) * PR...,NaN
3,Colonization,"Colonization of ""healthy"" host by DR without i...",M_M,M_DR,(beta * (1 - CR_N) * (1 - p_beta_infection_M) ...,NaN
4,Colonization,"Colonization of ""healthy"" host by DR with inst...",M_M,DR_DR,(beta * (1 - CR_N) * (p_beta_infection_M) * PR...,NaN
5,Colonization,Colonization of low dysbiosis host by DS witho...,M_X,M_DS,(beta * (1 - p_beta_infection_M) * PRESSURE_DS...,NaN
...,...,...,...,...,...,...
57,Recovery,Spontaneous decolonization via recovery,M_DR,M_M,gammaN0,NaN
58,Recovery,Spontaneous decolonization via recovery,M_DSDR,M_M,gammaN0,NaN
59,Recovery,Background recovery of commensal diversity,M_X,M_M,gN0,NaN
60,Competition,DS wins out over DR,*_DSDR,*_DS,omega,NaN


## Parse file into a Model

In [7]:
# PROBLEM: DSDR can't exist in high abundance
families = {
    'star': ['M', 'X', 'DS', 'DR', 'DSDR']
}

In [8]:
def parse_arrow(arrow):
    family_flag = False

    if pd.isna(arrow.trigger):
        arrow_dictionary = {'k': arrow.k}
    else:
        arrow_dictionary = {'p': arrow.k}

    reactants = arrow.Initial
    if reactants.count('*') > 1: assert False
    elif reactants.count('*') == 1: family_flag = True
    reactants = reactants.replace('*', '$i')

    products = arrow.Final
    if products.count('*') > 1: assert False
    elif products.count('*') == 1: family_flag = True
    products = products.replace('*', '$i')

    reactants = [reactants] if reactants else []
    products = [products] if products else []

    arrow_dictionary.update({
        'description': arrow.Description,
        'reactants': reactants,
        'products': products,
    })

    if family_flag:
        arrow_dictionary['used_families'] = {'i': 'star'}

    return arrow_dictionary

In [9]:
arrows_file.parse('Hooks', index_col='Triggering group').loc['Discharge']['Triggered group']

'Admission'

In [ ]:
from collections import defaultdict
reactions = []
triggered_sets = defaultdict(lambda: [])

for i, arrow in arrows.iterrows():
    destination = reactions
    if not pd.isna(arrow.trigger):
        destination = triggered_sets[arrow.Group]

    arrow_dictionary = parse_arrow(arrow)
    destination.append(arrow_dictionary)

M_M , M_DS
{'k': '(beta * (1 - CR_N) * (1 - p_beta_infection_M) * PRESSURE_DS)*((1 - CR_A))', 'description': 'Colonization of "healthy" host by DS without instant infection', 'reactants': ['M_M'], 'products': ['M_DS']}
M_M , DS_DS
{'k': '(beta * (1 - CR_N) * (p_beta_infection_M) * PRESSURE_DS )*((1 - CR_A))', 'description': 'Colonization of "healthy" host by DS with instant infection', 'reactants': ['M_M'], 'products': ['DS_DS']}
M_M , M_DR
{'k': '(beta * (1 - CR_N) * (1 - p_beta_infection_M) * PRESSURE_DR)*((1 - CR_A))', 'description': 'Colonization of "healthy" host by DR without instant infection', 'reactants': ['M_M'], 'products': ['M_DR']}
M_M , DR_DR
{'k': '(beta * (1 - CR_N) * (p_beta_infection_M) * PRESSURE_DR)*((1 - CR_A))', 'description': 'Colonization of "healthy" host by DR with instant infection', 'reactants': ['M_M'], 'products': ['DR_DR']}
M_X , M_DS
{'k': '(beta * (1 - p_beta_infection_M) * PRESSURE_DS)*((1 - CR_A))', 'description': 'Colonization of low dysbiosis host b

In [ ]:
species_df = arrows_file.parse('Species')
base_species = [Species(s.Name, description=s.Description) for _,s in species_df.fillna('').iterrows()]

bad_species = [
    {'name':'DS_M'},
    {'name':'DS_X'},
    {'name':'DS_DR'},
    {'name':'DR_M'},
    {'name':'DR_X'},
    {'name':'DR_DS'},
    {'name':'X_M'},
    {'name':'DSDR_DSDR'},
    {'name':'DSDR_DS'},
    {'name':'DSDR_DR'},
]
bad_species = [Species(**s) for s in bad_species]

species = [s.to_dict() for s in base_species] + [s.to_dict() for s in bad_species]



In [ ]:
model_dictionary = {
    'families': families,
    'species': species,
    'reactions': reactions,
    'triggered_sets': triggered_sets
}

results = parser.loads(model_dictionary, model_class=HookAwareModel)

In [83]:
filtered_reactions = [r for r in results.model.all_reactions if not set(r.reactants).intersection(bad_species) and not set(r.products).intersection(bad_species)]

In [80]:
filtered_model = HookAwareModel(
    base_species,
    [r for r in results.model.all_reactions if not set(r.reactants).intersection(bad_species) and not set(r.products).intersection(bad_species)]
)